In [9]:
import numpy as np
import pandas as pd
import sklearn

import keras
from keras.preprocessing import image

from tqdm import tqdm

In [10]:
movies = pd.read_csv('archive/movies_metadata.csv')
print("Shape: ", movies.shape)
print("Features: ", movies.columns)

Shape:  (45466, 24)
Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


/Users/anshverma/opt/anaconda3/envs/python37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
movies.head()['imdb_id']

0    tt0114709
1    tt0113497
2    tt0113228
3    tt0114885
4    tt0113041
Name: imdb_id, dtype: object

In [12]:
movies.head()['genres']

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

In [13]:
movies.head()['poster_path']

0    /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1    /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2    /6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3    /16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4    /e64sOI48hQXyru7naBFyssKFxVd.jpg
Name: poster_path, dtype: object

In [14]:
movies.head()['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [15]:
movies.head()['genres'][1]

"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"

In [16]:
# ARCHIVED FUNCTION
"""
def f(downloaded_image_dir, title, poster_path):
    if not os.path.exists(downloaded_image_dir):
        os.makedirs(downloaded_image_dir)
    
    # corrupted url
    imgUrl = "https://www.themoviedb.org/t/p/w1280" + str(poster_path)
    filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'

    try:
        r = requests.get(imgUrl, stream=True)
        if r.status_code == 200:
            with open(downloaded_image_dir + '/' + filename, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
        else:
            print("Not Found: ", title, imgUrl)
        
    except:
        print("Error Dowloading: ", title, imgUrl)    
"""

'\ndef f(downloaded_image_dir, title, poster_path):\n    if not os.path.exists(downloaded_image_dir):\n        os.makedirs(downloaded_image_dir)\n    \n    # corrupted url\n    imgUrl = "https://www.themoviedb.org/t/p/w1280" + str(poster_path)\n    filename = re.sub(r\'\\W+\', \' \', title).lower().strip().replace(" ", "-") + \'.jpg\'\n\n    try:\n        r = requests.get(imgUrl, stream=True)\n        if r.status_code == 200:\n            with open(downloaded_image_dir + \'/\' + filename, \'wb\') as f:\n                r.raw.decode_content = True\n                shutil.copyfileobj(r.raw, f)\n        else:\n            print("Not Found: ", title, imgUrl)\n        \n    except:\n        print("Error Dowloading: ", title, imgUrl)    \n'

In [19]:
# @Source: https://johannesbader.ch/blog/tutorial-download-posters-with-the-movie-database-api-in-python/"

import os
import re
import requests

CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'
IMG_PATTERN = 'http://api.themoviedb.org/3/movie/{imdbid}/images?api_key={key}' 
KEY = '6f2ceb0e0b459afe90f6e854fcd410eb'
            
def _get_json(url):
    r = requests.get(url)
    return r.json()
    
def _download_images(titles, urls, path):
    """download all images in list 'urls' to 'path' """
    print(len(urls), len(titles))
    zipped = list(zip(titles, urls))
    for title, url in zipped:
        r = requests.get(url)
        filetype = r.headers['content-type'].split('/')[-1]
        filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'
        filepath = os.path.join(path, filename)
        
        if r.status_code == 200:
            with open(filepath,'wb') as w:
                w.write(r.content)

def get_poster_url(imdbid):
    """ 
    given IMDB id, returns the corresponding url for
    poster jpg
    
    Args: 
        imdbid (str): IMDB id
    Return: 
        url (str): image url of poster image from 'themoviedb.org'
                of the maximum available size. 
    """
    config = _get_json(CONFIG_PATTERN.format(key=KEY))
    base_url = config['images']['base_url']
    sizes = config['images']['poster_sizes']   # sorted in ascending order
    
    def size_str_to_int(x):
        return float("inf") if x == 'original' else int(x[1:])
    
    # retrieve image of max size
    max_size = max(sizes, key=size_str_to_int)
    
    posters = _get_json(IMG_PATTERN.format(key=KEY,imdbid=imdbid))['posters']
    
    if len(posters) == 0:
        return None
    
    rel_path = posters[0]['file_path']
    url = "{0}{1}{2}".format(base_url, max_size, rel_path)
    
    return url

In [20]:
# RUN THIS FUNCTION TO DOWNLOAD JPG IMAGES
def tmdb_posters():
    """ download posters with imdb_id from dataset """
    
    if not os.path.exists('images_movies_genre'):
        os.makedirs('images_movies_genre')
    
    titles = []
    urls = []
    
    n = 0
    for index, row in movies.iterrows():
        title = str(row['title'])
        url = get_poster_url(row['imdb_id'])
        
        # skip iteration if poster cannot be located
        if not url:
            continue
            
        titles.append(title)
        urls.append(url)
        n += 1
        
        # SET LIMIT FOR NUMBER OF IMAGES TO DOWNLOAD
        if n > 20:
            break

    _download_images(titles, urls, 'images_movies_genre')

tmdb_posters() 

21 21


In [21]:
# TODO: jpg --> numpy array conversion

X = []

for i in tqdm(range(movies.shape[0])):
    img = image.load_img(path, target_size=(350, 350, 3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
    
X = np.array(X)

  0%|          | 0/45466 [00:00<?, ?it/s]


NameError: name 'path' is not defined

In [22]:
# TODO: Padding
# reference here: https://jinglescode.github.io/2019/11/17/predict-movie-earnings-with-posters/

# Create a larger zero canvas
canvas = np.zeros(512, 512, 3)

# Overlay with image
canvas[6:506, 6:506] = your_500_500_img

TypeError: Cannot interpret '512' as a data type

In [23]:
import ast
#reference-https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
def genre_list(str):
    arr=ast.literal_eval(str)
    genreList=[dict['name'] for dict in arr]
    return genreList

In [24]:
movies['genres'].isnull().values.any()

False

In [25]:
genre_list_collection=[]
for i in movies['genres']:
    genre_list_collection+=[genre_list(i)]

In [26]:
#adding a column called genre_list which contains all the genres of the movies in a list
movies.insert(5,"genre_list", genre_list_collection, True)
print("Features: ", movies.columns)
movies['genre_list']

Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'genre_list', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genre_list, Length: 45466, dtype: object

In [41]:
"""Removing genres that are very sparse in dataset"""

""" archived code because unwanted genres already decided below
dictSet = {}
totGenres = 0
for movie in movies['genre_list']: 
    for genre in movie: 
        if dictSet.get(genre, 0): 
            dictSet[genre] += 1
        else: 
            dictSet[genre] = 1
        totGenres += 1

dictSetIter = dictSet.copy()
for genre in dictSetIter: 
    if not (dictSet[genre]/(totGenres) < 0.01):
        print(genre)
        dictSet.pop(genre)

print()

for movie in movies['genre_list']: 
    movieIter = movie.copy()
    for genre in movieIter: 
        if dictSet.get(genre, 0):
            movie.remove(genre)
            print(genre)
"""

" archived code because unwanted genres already decided below\ndictSet = {}\ntotGenres = 0\nfor movie in movies['genre_list']: \n    for genre in movie: \n        if dictSet.get(genre, 0): \n            dictSet[genre] += 1\n        else: \n            dictSet[genre] = 1\n        totGenres += 1\n\ndictSetIter = dictSet.copy()\nfor genre in dictSetIter: \n    if not (dictSet[genre]/(totGenres) < 0.01):\n        print(genre)\n        dictSet.pop(genre)\n\nprint()\n\nfor movie in movies['genre_list']: \n    movieIter = movie.copy()\n    for genre in movieIter: \n        if dictSet.get(genre, 0):\n            movie.remove(genre)\n            print(genre)\n"

In [43]:
#TOD0: Remove unwanted genres
invalid_genres = ['Aniplex', 'BROSTA TV', 'Carousel Productions', 'GoHands', 'Mardock Scramble Production Committee', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Sentai Filmworks', 'Telescene Film Group Productions', 'The Cartel', 'Vision View Entertainment', 'TV Movie', 'Foreign']
invalid_genres_dict = {}
for genre in invalid_genres: 
    invalid_genres_dict[genre] = 0
    
for movie in movies['genre_list']: 
    for genre in movie: 
        if invalid_genres_dict.get(genre, 0):
            movie.remove(genre)
            print(genre)

In [39]:
#One hot encoding the genres
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform(movies['genre_list'])
#TODO Add the one hot encoded genres to the movies dataframe

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
mlb.classes_

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'Thriller', 'War', 'Western'], dtype=object)

In [30]:
movies['genre_list']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genre_list, Length: 45466, dtype: object